# 1次元有限要素法における２次要素の剛性行列の確認

１次元有限要素法の２次要素の剛性行列が必要になったためGetfem++で作成しました。

In [1]:
import getfem as gf
import numpy as np

メッシュ作成の際に、'regular simplices'オプションで次数を２次にします。

In [2]:
m = gf.Mesh('regular simplices', np.arange(0, 2, 1),'degree',2)
print m



BEGIN POINTS LIST

  POINT  0  0
  POINT  1  0.5
  POINT  2  1

END POINTS LIST



BEGIN MESH STRUCTURE DESCRIPTION

CONVEX 0    'GT_PK(1,2)'      0  1  2

END MESH STRUCTURE DESCRIPTION



変位用オブジェクトとデータ用オブジェクトを作成します。

In [3]:
mfu = gf.MeshFem(m, 1)
mfu.set_fem(gf.Fem('FEM_PK(1,2)'))
print mfu

mfd = gf.MeshFem(m, 1)
mfd.set_fem(gf.Fem('FEM_QK(1,2)'))
print mfd


BEGIN MESH_FEM

QDIM 1
 CONVEX 0 'FEM_PK(1,2)'
 BEGIN DOF_ENUMERATION 
  0:  0 1 2
 END DOF_ENUMERATION 
END MESH_FEM


BEGIN MESH_FEM

QDIM 1
 CONVEX 0 'FEM_PK(1,2)'
 BEGIN DOF_ENUMERATION 
  0:  0 1 2
 END DOF_ENUMERATION 
END MESH_FEM



積分法は'GAUSS1D'の２次とします。

In [4]:
mim = gf.MeshIm(m, gf.Integ('IM_GAUSS1D(2)'))
print mim


BEGIN MESH_IM

 CONVEX 0 'IM_GAUSS1D(2)'
END MESH_IM



剛性行列は以下の様になりました。

In [5]:
K = gf.asm_linear_elasticity(mim, mfu, mfd, np.repeat([1.000E+00],3), np.repeat([1.000E+00],3))
K.full()

array([[  7.,  -8.,   1.],
       [ -8.,  16.,  -8.],
       [  1.,  -8.,   7.]])